# PopSim Data Preparation (Census Mode)

This notebook automates population synthesis setup using German Census grid data.

## What You Need

1. **A GeoPackage** (`.gpkg`) defining your study area boundary
2. **Census parquet files** in the `data/` folder:
   - `cells_100m_*.parquet` - 100m grid data
   - `cells_1km_*.parquet` - 1km grid data

## Census Data Format

Any parquet file works as "census" data as long as the **first column** contains cell IDs in this format:
- **100m**: `CRS3035RES100mN{northing}E{easting}` (e.g., `CRS3035RES100mN2689100E4337000`)
- **1km**: `CRS3035RES1000mN{northing}E{easting}` (e.g., `CRS3035RES1000mN2689000E4337000`)

The coordinates are in **EPSG:3035** (ETRS89-extended / LAEA Europe). All other columns become available as control totals.

## What Gets Automated

- Census cells are automatically filtered to your study area (via GeoPackage)
- Geographic crosswalk (100m → 1km → STAAT) is auto-generated
- Control totals are auto-populated from census columns
- You only need to define control expressions

## Configuration

**Edit these settings before running.**

### CSV Separator Notes
- `census_csv_sep`: Separator for **input** CSV files (ignored for parquet). Use `;` for German/European data.
- `intermediate_sep`: Separator for intermediate files (`_prep3_controls.csv`). Use `;` if editing in Excel with German locale.
- **Final files** (consumed by PopSim) are **always comma-separated** - this is required for PopSim compatibility.

In [ ]:
# =============================================================================
# USER CONFIGURATION
# =============================================================================

# Path to your study area GeoPackage
geopackage_path = "data/outlineNIwithBremen.gpkg"

# GeoPackage CRS (set to None to auto-detect, or specify e.g., "EPSG:25832")
geopackage_crs = "EPSG:25832"  # UTM 32N - common for German data

# Census data files (parquet or CSV)
census_100m_path = "data/cells_100m_with_gender_backf_binneds_happyorphans.parquet"
census_1km_path = "data/cells_1km_with_binneds.parquet"

# CSV settings (ignored for parquet files)
census_csv_sep = ";"  # Separator for input CSV files: "," or ";"

# Intermediate file separator (for _prep*, _census_filtered, etc.)
# Use ";" if your locale uses comma as decimal separator
intermediate_sep = ","  # "," or ";" - finals are always ","

# Column to use as number of households (from census data)
# Set to None to see available columns, then set and re-run
household_column = None  # e.g., "Insgesamt_Haushalte_100m-Gitter"

# Output settings
output_everything = False  # Set True to output all intermediate PopSim files

# Geography settings
seed_geography = "STAAT"  # Usually unchanged

# MiD (seed data) filtering
filter_mid = False
kernwo = [2, 3]  # Day of week filter (list): 1=Monday, 2=Tue-Thu, 3=Friday, 4=Sat-Sun
regiostar17 = [121, 123, 124]  # Regional types to include

# =============================================================================
# END CONFIGURATION
# =============================================================================

## Step 1: Load Study Area and Filter Census

Loads your GeoPackage, filters census cells to the study area, and shows available columns.

In [ ]:
import json
import os
import re
import pandas as pd
import geopandas as gpd
from shapely.geometry import box

print("[Step 1/4] Loading study area and filtering census...")
print("=" * 60)

# Helper to load census file (parquet or CSV)
def load_census_file(path, csv_sep=",", batch_mode=False):
    """Load census data from parquet or CSV."""
    if path.endswith('.parquet'):
        if batch_mode:
            import pyarrow.parquet as pq
            return pq.ParquetFile(path)
        return pd.read_parquet(path)
    elif path.endswith('.csv'):
        return pd.read_csv(path, sep=csv_sep)
    else:
        raise ValueError(f"Unsupported file format: {path}. Use .parquet or .csv")

# Load GeoPackage
print(f"Loading GeoPackage: {geopackage_path}")
study_area = gpd.read_file(geopackage_path)

# Handle CRS
if study_area.crs is None and geopackage_crs:
    study_area = study_area.set_crs(geopackage_crs)
    print(f"  Set CRS to: {geopackage_crs}")
elif study_area.crs is None:
    raise ValueError("GeoPackage has no CRS. Please set geopackage_crs in configuration.")

# Transform to EPSG:3035 (Census CRS)
study_area_3035 = study_area.to_crs("EPSG:3035")
bounds = study_area_3035.total_bounds  # minx, miny, maxx, maxy
print(f"  Study area bounds (EPSG:3035): {bounds}")

# Parse cell ID to extract coordinates
def parse_cell_id_100m(cell_id):
    """Extract N,E coordinates from 100m cell ID like CRS3035RES100mN2689100E4337000"""
    match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(cell_id))
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

def parse_cell_id_1km(cell_id):
    """Extract N,E coordinates from 1km cell ID like CRS3035RES1000mN2689000E4337000"""
    match = re.match(r'CRS3035RES1000mN(\d+)E(\d+)', str(cell_id))
    if match:
        return int(match.group(1)), int(match.group(2))
    return None, None

def get_1km_id_from_100m(cell_id_100m):
    """Convert 100m cell ID to corresponding 1km cell ID."""
    n, e = parse_cell_id_100m(cell_id_100m)
    if n is None:
        return None
    # Round down to nearest 1000m
    n_1km = (n // 1000) * 1000
    e_1km = (e // 1000) * 1000
    return f"CRS3035RES1000mN{n_1km}E{e_1km}"

# Load 100m census
print(f"\nLoading 100m census: {census_100m_path}")

if census_100m_path.endswith('.parquet'):
    import pyarrow.parquet as pq
    pf_100m = pq.ParquetFile(census_100m_path)
    print(f"  Total rows: {pf_100m.metadata.num_rows:,}")
    print(f"  Total columns: {pf_100m.metadata.num_columns}")
    
    # Read in batches to filter efficiently
    print("  Filtering to study area (this may take a moment)...")
    filtered_chunks = []
    total_read = 0
    
    for batch in pf_100m.iter_batches(batch_size=100000):
        df_batch = batch.to_pandas()
        total_read += len(df_batch)
        
        # Parse coordinates and filter
        coords = df_batch.iloc[:, 0].apply(parse_cell_id_100m)
        df_batch['_N'] = coords.apply(lambda x: x[0])
        df_batch['_E'] = coords.apply(lambda x: x[1])
        
        # Bounding box filter
        mask = (
            (df_batch['_N'] >= bounds[1]) & (df_batch['_N'] <= bounds[3]) &
            (df_batch['_E'] >= bounds[0]) & (df_batch['_E'] <= bounds[2])
        )
        df_filtered = df_batch[mask].drop(columns=['_N', '_E'])
        
        if len(df_filtered) > 0:
            filtered_chunks.append(df_filtered)
        
        if total_read % 500000 == 0:
            print(f"    Processed {total_read:,} rows...")
    
    census_100m = pd.concat(filtered_chunks, ignore_index=True)
else:
    # CSV - load all at once (usually smaller files)
    print(f"  Loading CSV with separator: '{census_csv_sep}'")
    census_100m_full = pd.read_csv(census_100m_path, sep=census_csv_sep)
    print(f"  Total rows: {len(census_100m_full):,}")
    
    # Filter by bounding box
    coords = census_100m_full.iloc[:, 0].apply(parse_cell_id_100m)
    census_100m_full['_N'] = coords.apply(lambda x: x[0])
    census_100m_full['_E'] = coords.apply(lambda x: x[1])
    
    mask = (
        (census_100m_full['_N'] >= bounds[1]) & (census_100m_full['_N'] <= bounds[3]) &
        (census_100m_full['_E'] >= bounds[0]) & (census_100m_full['_E'] <= bounds[2])
    )
    census_100m = census_100m_full[mask].drop(columns=['_N', '_E']).copy()

print(f"  Filtered to {len(census_100m):,} cells in bounding box")

# Fine filter: check actual intersection with study area polygon
print("  Performing precise polygon intersection...")
id_col = census_100m.columns[0]

def cell_intersects_study_area(cell_id):
    n, e = parse_cell_id_100m(cell_id)
    if n is None:
        return False
    cell_geom = box(e, n, e + 100, n + 100)
    return study_area_3035.geometry.intersects(cell_geom).any()

# Sample check - if bbox is tight, skip full intersection
sample_mask = census_100m[id_col].sample(min(100, len(census_100m))).apply(cell_intersects_study_area)
if sample_mask.mean() > 0.9:
    print("  Bounding box is tight, skipping detailed intersection.")
else:
    mask = census_100m[id_col].apply(cell_intersects_study_area)
    census_100m = census_100m[mask]
    print(f"  After polygon intersection: {len(census_100m):,} cells")

# Show available columns for household selection
print(f"\n{'='*60}")
print("AVAILABLE COLUMNS IN 100m CENSUS DATA:")
print(f"{'='*60}")

# Find likely household columns
hh_keywords = ['haushalt', 'household', 'hh_', 'wohnung']
suggested = []
for col in census_100m.columns:
    col_lower = col.lower()
    if any(kw in col_lower for kw in hh_keywords):
        suggested.append(col)

if suggested:
    print("\nSUGGESTED HOUSEHOLD COLUMNS:")
    for col in suggested[:10]:
        print(f"  -> {col}")

print(f"\nALL COLUMNS ({len(census_100m.columns)}):")
for i, col in enumerate(census_100m.columns):
    if i < 30:
        print(f"  {col}")
    elif i == 30:
        print(f"  ... and {len(census_100m.columns) - 30} more")
        break

print(f"\n{'='*60}")
print(f"DATA PREVIEW (first 3 rows, first 5 columns):")
print(f"{'='*60}")
print(census_100m.iloc[:3, :5].to_string())

# Load 1km census
print(f"\n{'='*60}")
print(f"Loading 1km census: {census_1km_path}")

if census_1km_path.endswith('.parquet'):
    census_1km_full = pd.read_parquet(census_1km_path)
else:
    census_1km_full = pd.read_csv(census_1km_path, sep=census_csv_sep)
print(f"  Total rows: {len(census_1km_full):,}")

# Filter 1km by deriving from 100m cells
km_ids_needed = set(census_100m[id_col].apply(get_1km_id_from_100m).dropna())
km_id_col = census_1km_full.columns[1]  # Usually GITTER_ID_1km
census_1km = census_1km_full[census_1km_full[km_id_col].isin(km_ids_needed)].copy()
print(f"  Filtered to {len(census_1km):,} 1km cells")

# Save filtered data as parquet (always - most efficient for intermediates)
census_100m.to_parquet('data/_census_100m_filtered.parquet', index=False)
census_1km.to_parquet('data/_census_1km_filtered.parquet', index=False)
print(f"\nSaved filtered census to data/_census_*_filtered.parquet")

# Normalize kernwo to list
kernwo_list = kernwo if isinstance(kernwo, list) else [kernwo]

# Save config
config = {
    "geopackage_path": geopackage_path,
    "census_100m_id_col": id_col,
    "census_1km_id_col": km_id_col,
    "household_column": household_column,
    "output_everything": output_everything,
    "seed_geography": seed_geography,
    "filter_mid": filter_mid,
    "kernwo": kernwo_list,
    "regiostar17": regiostar17,
    "intermediate_sep": intermediate_sep,
    "num_100m_cells": len(census_100m),
    "num_1km_cells": len(census_1km),
}
with open("prep_config.json", "w") as f:
    json.dump(config, f, indent=2)

print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
print(f"  100m cells in study area: {len(census_100m):,}")
print(f"  1km cells in study area: {len(census_1km):,}")
print(f"  Intermediate file separator: '{intermediate_sep}'")
print(f"\nSet 'household_column' in Configuration cell and re-run Step 1,")
print("or proceed to Step 2 if already set.")
print("\n[Step 1/4] Complete.")

## Step 2: Generate Geo Crosswalk and Control Totals

Automatically creates:
- `geo_cross_walk.csv` - mapping 100m → 1km → STAAT
- `control_totals_*.csv` - census data as control totals

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import re
import yaml
from unidecode import unidecode

print("[Step 2/4] Generating geo crosswalk and control totals...")
print("=" * 60)

# Load config
with open("prep_config.json", "r") as f:
    config = json.load(f)

household_column = config["household_column"]
id_col_100m = config["census_100m_id_col"]
id_col_1km = config["census_1km_id_col"]
output_everything = config["output_everything"]
seed_geography = config["seed_geography"]
intermediate_sep = config.get("intermediate_sep", ",")  # Default to comma

if household_column is None:
    raise ValueError("household_column not set! Set it in Configuration and re-run Step 1.")

# Load filtered census
census_100m = pd.read_parquet('data/_census_100m_filtered.parquet')
census_1km = pd.read_parquet('data/_census_1km_filtered.parquet')

print(f"Loaded {len(census_100m):,} 100m cells, {len(census_1km):,} 1km cells")

# Validate household column
if household_column not in census_100m.columns:
    raise ValueError(f"household_column '{household_column}' not found in census data.")

# Check household values
hh_values = census_100m[household_column]
if hh_values.isna().any():
    na_count = hh_values.isna().sum()
    print(f"WARNING: {na_count} cells have missing household values (will be set to 0)")
if (hh_values < 0).any():
    neg_count = (hh_values < 0).sum()
    print(f"WARNING: {neg_count} cells have negative household values")

# Helper to get 1km ID from 100m ID
def get_1km_from_100m(cell_id):
    """Convert 100m cell ID to corresponding 1km cell ID."""
    match = re.match(r'CRS3035RES100mN(\d+)E(\d+)', str(cell_id))
    if match:
        n, e = int(match.group(1)), int(match.group(2))
        # Round down to nearest 1000m
        n_1km = (n // 1000) * 1000
        e_1km = (e // 1000) * 1000
        return f"CRS3035RES1000mN{n_1km}E{e_1km}"
    return None

# Standardize column names
def clean_col_name(name):
    return unidecode(name).replace(" ", "").replace(".", "").replace(",", "").replace("-", "_")

# Rename columns
census_100m.columns = [clean_col_name(c) for c in census_100m.columns]
census_1km.columns = [clean_col_name(c) for c in census_1km.columns]
household_column_clean = clean_col_name(household_column)

# Find the ID columns after cleaning
id_col_100m_clean = census_100m.columns[0]
id_col_1km_clean = [c for c in census_1km.columns if '1km' in c.lower()][0]

# Create geo_cross_walk (hierarchy: ZENSUS100m -> ZENSUS1km -> STAAT -> WELT)
print("\nCreating geo_cross_walk...")
geo_cross = pd.DataFrame()
geo_cross['ZENSUS100m'] = census_100m[id_col_100m_clean]
geo_cross['ZENSUS1km'] = geo_cross['ZENSUS100m'].apply(get_1km_from_100m)
geo_cross['STAAT'] = 1
geo_cross['WELT'] = 1

geo_cross.to_csv('data/geo_cross_walk.csv', index=False)  # Final - always comma
print(f"  Created data/geo_cross_walk.csv ({len(geo_cross)} rows)")

# Create control_totals for 100m (lowest level)
print("\nCreating control totals...")

# Geography names (hierarchy from lowest to highest)
geo_names = ['ZENSUS100m', 'ZENSUS1km', 'STAAT', 'WELT']

# Rename household column to numberOfHouseholds
census_100m = census_100m.rename(columns={household_column_clean: 'numberOfHouseholds'})

# Add geography columns
census_100m = census_100m.rename(columns={id_col_100m_clean: 'ZENSUS100m'})
census_100m['ZENSUS1km'] = census_100m['ZENSUS100m'].apply(get_1km_from_100m)
census_100m['STAAT'] = 1
census_100m['WELT'] = 1

# Suffix non-geo columns
for col in census_100m.columns:
    if col not in geo_names:
        census_100m.rename(columns={col: f"{col}_ZENSUS100m"}, inplace=True)

census_100m = census_100m.fillna(0)
census_100m.to_csv('data/control_totals_ZENSUS100m.csv', index=False)  # Final - always comma
print(f"  Created data/control_totals_ZENSUS100m.csv")

# Create control_totals for 1km
census_1km = census_1km.rename(columns={id_col_1km_clean: 'ZENSUS1km'})
census_1km['STAAT'] = 1
census_1km['WELT'] = 1

for col in census_1km.columns:
    if col not in geo_names:
        census_1km.rename(columns={col: f"{col}_ZENSUS1km"}, inplace=True)

census_1km = census_1km.fillna(0)
census_1km.to_csv('data/control_totals_ZENSUS1km.csv', index=False)  # Final - always comma
print(f"  Created data/control_totals_ZENSUS1km.csv")

# Create control_totals for STAAT
staat_df = pd.DataFrame({'STAAT': [1], 'WELT': [1]})
staat_df.to_csv('data/control_totals_STAAT.csv', index=False)  # Final - always comma
print(f"  Created data/control_totals_STAAT.csv")

# Create control_totals for WELT (top level)
welt_df = pd.DataFrame({'WELT': [1]})
welt_df.to_csv('data/control_totals_WELT.csv', index=False)  # Final - always comma
print(f"  Created data/control_totals_WELT.csv")

# Create controls template
print("\nCreating controls template...")
controls_data = {
    'target': [],
    'geography': [],
    'seed_table': [],
    'importance': [],
    'control_field': [],
    'expression': []
}

total_hh_control = None

# Add 100m controls
for col in census_100m.columns:
    if col not in geo_names:
        controls_data['target'].append(f"{col}_target")
        controls_data['geography'].append('ZENSUS100m')
        controls_data['control_field'].append(col)
        if col.startswith('numberOfHouseholds'):
            total_hh_control = f"{col}_target"

# Add 1km controls
for col in census_1km.columns:
    if col not in geo_names:
        controls_data['target'].append(f"{col}_target")
        controls_data['geography'].append('ZENSUS1km')
        controls_data['control_field'].append(col)

# Add example expression
controls_data['expression'].append('(households.H_GEW > 0) & (households.H_GEW < np.inf)')

controls_df = pd.DataFrame({k: pd.Series(v) for k, v in controls_data.items()})
# Intermediate file - use configured separator
controls_df.to_csv('configs/_prep3_controls.csv', index=False, sep=intermediate_sep)
print(f"  Created configs/_prep3_controls.csv ({len(controls_df)} controls, sep='{intermediate_sep}')")

if total_hh_control is None:
    raise ValueError("Could not find numberOfHouseholds control!")

# Update settings.yaml
print("\nUpdating PopSim configuration...")
with open('configs/settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)

# Geographies from top to bottom: WELT -> STAAT -> ZENSUS1km -> ZENSUS100m
settings['geographies'] = ['WELT', 'STAAT', 'ZENSUS1km', 'ZENSUS100m']
settings['seed_geography'] = seed_geography
settings['total_hh_control'] = total_hh_control

# Update input tables
idx = next((i for i, t in enumerate(settings['input_table_list']) if t['tablename'] == 'geo_cross_walk'), None)
if idx is not None:
    settings['input_table_list'] = settings['input_table_list'][:idx + 1]

for geo in ['ZENSUS100m', 'ZENSUS1km', 'STAAT', 'WELT']:
    settings['input_table_list'].append({
        'tablename': f'{geo}_control_data',
        'filename': f'control_totals_{geo}.csv'
    })

# Update output tables
if output_everything:
    settings['output_tables'] = {'action': 'skip', 'tables': 'geo_cross_walk'}
else:
    settings['output_tables'] = {
        'action': 'include',
        'tables': ['expanded_household_ids', 
                   'summary_ZENSUS100m', 'summary_ZENSUS1km', 'summary_STAAT', 'summary_WELT',
                   f'summary_ZENSUS100m_{seed_geography}']
    }

# Update models
settings['models'] = [m for m in settings['models'] if 'sub_balancing' not in m]
idx = settings['models'].index('integerize_final_seed_weights')
settings['models'].insert(idx + 1, 'sub_balancing.geography=ZENSUS100m')

with open('configs/settings.yaml', 'w') as f:
    yaml.dump(settings, f, default_flow_style=False)
print("  Updated configs/settings.yaml")

# Update verification.yaml
with open('scripts/verification.yaml', 'r') as f:
    verify = yaml.safe_load(f)

verify['group_geographies'] = ['WELT', 'STAAT', 'ZENSUS1km', 'ZENSUS100m']
verify['seed_cols']['geog'] = seed_geography
verify['summaries'] = [
    'output/final_summary_ZENSUS100m.csv',
    'output/final_summary_ZENSUS1km.csv',
    'output/final_summary_STAAT.csv',
    'output/final_summary_WELT.csv',
    f'output/final_summary_ZENSUS100m_{seed_geography}.csv'
]

with open('scripts/verification.yaml', 'w') as f:
    yaml.dump(verify, f, default_flow_style=False)
print("  Updated scripts/verification.yaml")

# Update config
config['geo_names'] = geo_names
config['total_hh_control'] = total_hh_control
with open('prep_config.json', 'w') as f:
    json.dump(config, f, indent=2)

# Summary
print(f"\n{'='*60}")
print("SUMMARY")
print(f"{'='*60}")
hh_col = 'numberOfHouseholds_ZENSUS100m'
total_hh = census_100m[hh_col].sum()
print(f"  Geographic hierarchy: WELT -> STAAT -> ZENSUS1km -> ZENSUS100m")
print(f"  Geographic units: {len(census_100m):,} (100m), {len(census_1km):,} (1km)")
print(f"  Total households: {total_hh:,.0f}")
print(f"  Controls defined: {len(controls_df)}")
print(f"  Intermediate separator: '{intermediate_sep}'")
print(f"\nNext: Edit configs/_prep3_controls.csv to add expressions for controls you want.")
print("\n[Step 2/4] Complete.")

## Step 3: Edit Controls and Process Seed Data

Edit `configs/_prep3_controls.csv` to add expressions for the controls you want to use, then run this cell.

In [ ]:
import pandas as pd
import os
import json
import re
import math

print("[Step 3/4] Processing seed data...")
print("=" * 60)

# Load config
with open("prep_config.json", "r") as f:
    config = json.load(f)

filter_mid = config["filter_mid"]
kernwo = config["kernwo"]  # Now a list
regiostar17 = config["regiostar17"]
intermediate_sep = config.get("intermediate_sep", ",")  # Default to comma

# Load controls (intermediate file - use configured separator)
print(f"Loading controls template (separator: '{intermediate_sep}')...")
controls_df = pd.read_csv('configs/_prep3_controls.csv', sep=intermediate_sep)
print(f"  Loaded {len(controls_df)} controls from _prep3_controls.csv")

# Load seed data (MiD files always use semicolon - German standard)
print("\nLoading MiD seed data...")
seed_persons = pd.read_csv('data/MiD2017_Personen.csv', sep=';')
seed_households = pd.read_csv('data/MiD2017_Haushalte.csv', sep=';')

print(f"  Loaded {len(seed_persons):,} persons, {len(seed_households):,} households")

if filter_mid:
    print(f"\nApplying MiD filters:")
    persons_before = len(seed_persons)
    households_before = len(seed_households)
    
    # Filter by kernwo (day of week) - applies to persons
    if 'kernwo' in seed_persons.columns:
        seed_persons = seed_persons[seed_persons['kernwo'].isin(kernwo)]
        print(f"  kernwo filter {kernwo}: {persons_before:,} -> {len(seed_persons):,} persons")
    
    # Filter by RegioStaR17
    if 'RegioStaR17' in seed_persons.columns:
        seed_persons = seed_persons[seed_persons['RegioStaR17'].isin(regiostar17)]
    if 'RegioStaR17' in seed_households.columns:
        seed_households = seed_households[seed_households['RegioStaR17'].isin(regiostar17)]
    print(f"  RegioStaR17 filter {regiostar17}: {len(seed_persons):,} persons, {len(seed_households):,} households")

print(f"\nFinal counts:")
print(f"  Persons: {len(seed_persons):,}")
print(f"  Households: {len(seed_households):,}")

# Essential columns
essential_cols = {'H_ID', 'H_GEW', 'HP_ID', 'P_ID', 'P_GEW'}
needed_cols = essential_cols.copy()

# Extract columns from expressions
pattern = r'\.(?P<col>[A-Za-z_][A-Za-z0-9_]*)'
for expr in controls_df['expression'].dropna():
    for match in re.finditer(pattern, str(expr)):
        needed_cols.add(match.group('col'))

print(f"\nColumns needed from expressions: {needed_cols - essential_cols}")

# Filter to needed columns
p_cols = list(needed_cols.intersection(seed_persons.columns))
h_cols = list(needed_cols.intersection(seed_households.columns))

seed_persons = seed_persons[p_cols]
seed_households = seed_households[h_cols]

# Add STAAT geography
seed_persons['STAAT'] = 1
seed_households['STAAT'] = 1

# Save (final files - always comma separated for PopSim compatibility)
seed_persons.to_csv('data/seed_persons.csv', index=False)  # Final - always comma
seed_households.to_csv('data/seed_households.csv', index=False)  # Final - always comma
controls_df.to_csv('configs/controls.csv', index=False)  # Final - always comma

print(f"\nCreated (all comma-separated for PopSim):")
print(f"  data/seed_persons.csv ({len(seed_persons)} rows, {len(seed_persons.columns)} cols)")
print(f"  data/seed_households.csv ({len(seed_households)} rows, {len(seed_households.columns)} cols)")
print(f"  configs/controls.csv")

print("\n[Step 3/4] Complete.")

## Step 4: Validate and Run

Validates the setup and provides instructions for running PopSim.

In [ ]:
import os
import json
import yaml
import pandas as pd

print("[Step 4/4] Validating setup...")
print("=" * 60)

errors = []
warnings = []

# Check files
required_files = [
    'data/geo_cross_walk.csv',
    'data/seed_persons.csv',
    'data/seed_households.csv',
    'data/control_totals_ZENSUS100m.csv',
    'data/control_totals_ZENSUS1km.csv',
    'data/control_totals_STAAT.csv',
    'data/control_totals_WELT.csv',
    'configs/settings.yaml',
    'configs/controls.csv',
]

print("\nChecking files...")
for f in required_files:
    if os.path.exists(f):
        size = os.path.getsize(f)
        print(f"  [OK] {f} ({size:,} bytes)")
    else:
        print(f"  [MISSING] {f}")
        errors.append(f"Missing: {f}")

# Check controls
print("\nChecking controls...")
try:
    controls = pd.read_csv('configs/controls.csv')
    empty = controls['expression'].isna().sum()
    if empty > 0:
        errors.append(f"{empty} controls missing expressions")
    else:
        print(f"  {len(controls)} controls, all have expressions")
except Exception as e:
    errors.append(f"Error reading controls: {e}")

# Check settings
print("\nChecking settings.yaml...")
try:
    with open('configs/settings.yaml') as f:
        settings = yaml.safe_load(f)
    print(f"  Geographies: {settings.get('geographies')}")
    print(f"  Total HH control: {settings.get('total_hh_control')}")
except Exception as e:
    errors.append(f"Error reading settings: {e}")

# Summary
print(f"\n{'='*60}")
if errors:
    print("VALIDATION FAILED")
    for e in errors:
        print(f"  - {e}")
else:
    print("VALIDATION PASSED")
    print("\nReady to run PopSim:")
    print("  conda activate popsim")
    print("  python run_populationsim.py")

print(f"{'='*60}")
print("\n[Step 4/4] Complete.")

## Utilities: Reset

Clean up generated files to start fresh.

In [ ]:
import os

def reset(confirm=False):
    """Delete all generated files."""
    files = [
        'data/geo_cross_walk.csv',
        'data/seed_persons.csv',
        'data/seed_households.csv',
        'data/control_totals_ZENSUS100m.csv',
        'data/control_totals_ZENSUS1km.csv',
        'data/control_totals_STAAT.csv',
        'data/control_totals_WELT.csv',
        'data/_census_100m_filtered.parquet',
        'data/_census_1km_filtered.parquet',
        'configs/controls.csv',
        'configs/_prep3_controls.csv',
        'prep_config.json',
    ]
    
    existing = [f for f in files if os.path.exists(f)]
    
    if not existing:
        print("No files to delete.")
        return
    
    print("Files to delete:")
    for f in existing:
        print(f"  {f}")
    
    if not confirm:
        print("\nRun reset(confirm=True) to delete.")
        return
    
    for f in existing:
        os.remove(f)
        print(f"Deleted: {f}")
    print("\nReset complete.")

# Show what would be deleted
reset(confirm=False)